In [1]:
import pandas as pd
from pandas.tseries.holiday import USFederalHolidayCalendar
import datetime

In [2]:
from contextlib import contextmanager

def REFERENCE(solution, placeholder):
    return solution
@contextmanager
def REFBLOCK():
    yield

Helpful references:
* https://cs.calvin.edu/courses/data/202/ka37/topics/sql.html
* https://www.sqlite.org/lang_select.html
* https://www.sqlite.org/lang_corefunc.html

In [3]:
db_connection = "sqlite:///rides.db"

# Preprocess the data and create the SQL database

In [4]:
rides = pd.read_csv("2011-capitalbikeshare-tripdata.zip",
                    usecols=["Duration", "Start date", "End date", "Start station number", "End station number", "Member type"])
rides.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1226767 entries, 0 to 1226766
Data columns (total 6 columns):
Duration                1226767 non-null int64
Start date              1226767 non-null object
End date                1226767 non-null object
Start station number    1226767 non-null int64
End station number      1226767 non-null int64
Member type             1226767 non-null object
dtypes: int64(3), object(3)
memory usage: 56.2+ MB


In [5]:
# rename columns
rides = (
    rides
    .rename(columns=lambda x: x.lower().replace(' ', '_'))
    .rename(columns={'start_date': 'start_timestamp', 'end_date': 'end_timestamp'})
)

In [6]:
rides.head()

,duration,start_timestamp,end_timestamp,start_station_number,end_station_number,member_type
0,3548,2011-01-01 00:01:29,2011-01-01 01:00:37,31620,31620,Member
1,346,2011-01-01 00:02:46,2011-01-01 00:08:32,31105,31101,Casual
2,562,2011-01-01 00:06:13,2011-01-01 00:15:36,31400,31104,Member
3,434,2011-01-01 00:09:21,2011-01-01 00:16:36,31111,31503,Member
4,233,2011-01-01 00:28:26,2011-01-01 00:32:19,31104,31106,Casual


In [7]:
rides.to_sql("rides", db_connection, if_exists="replace", index=False)

In [8]:
# Run this code unchanged.
holidays = pd.DataFrame({
    'date': USFederalHolidayCalendar().holidays(datetime.date(2011,1,1), datetime.date(2015,12,31)).date,
    'is_holiday': True})
holidays.head()

,date,is_holiday
0,2011-01-17,True
1,2011-02-21,True
2,2011-05-30,True
3,2011-07-04,True
4,2011-09-05,True


In [9]:
holidays.to_sql("holidays", db_connection, if_exists="replace", index=False)

# `head`

In [10]:
rides.head(n=5) # Pandas way

,duration,start_timestamp,end_timestamp,start_station_number,end_station_number,member_type
0,3548,2011-01-01 00:01:29,2011-01-01 01:00:37,31620,31620,Member
1,346,2011-01-01 00:02:46,2011-01-01 00:08:32,31105,31101,Casual
2,562,2011-01-01 00:06:13,2011-01-01 00:15:36,31400,31104,Member
3,434,2011-01-01 00:09:21,2011-01-01 00:16:36,31111,31503,Member
4,233,2011-01-01 00:28:26,2011-01-01 00:32:19,31104,31106,Casual


In [11]:
pd.read_sql_query("""
    SELECT
      *
      FROM rides
      ORDER BY start_timestamp ASC
      LIMIT 5
""", db_connection)

,duration,start_timestamp,end_timestamp,start_station_number,end_station_number,member_type
0,3548,2011-01-01 00:01:29,2011-01-01 01:00:37,31620,31620,Member
1,346,2011-01-01 00:02:46,2011-01-01 00:08:32,31105,31101,Casual
2,562,2011-01-01 00:06:13,2011-01-01 00:15:36,31400,31104,Member
3,434,2011-01-01 00:09:21,2011-01-01 00:16:36,31111,31503,Member
4,233,2011-01-01 00:28:26,2011-01-01 00:32:19,31104,31106,Casual


## Exercise 1
Get the last 10 rides of the year.

**Note**: ordering options are "ASC" and "DESC".

In [12]:
# Pandas way:
rides.sort_values('start_timestamp', ascending=False).head(10)

,duration,start_timestamp,end_timestamp,start_station_number,end_station_number,member_type
1226766,468,2011-12-31 23:55:56,2012-01-01 00:03:45,31221,31111,Member
1226765,2060,2011-12-31 23:55:12,2012-01-01 00:29:33,31205,31222,Member
1226764,261,2011-12-31 23:47:27,2011-12-31 23:51:49,31107,31602,Member
1226763,387,2011-12-31 23:46:43,2011-12-31 23:53:10,31223,31201,Member
1226762,300,2011-12-31 23:41:19,2011-12-31 23:46:20,31201,31214,Member
1226761,349,2011-12-31 23:40:51,2011-12-31 23:46:40,31101,31203,Member
1226760,356,2011-12-31 23:40:39,2011-12-31 23:46:36,31101,31203,Member
1226759,410,2011-12-31 23:40:33,2011-12-31 23:47:23,31203,31214,Member
1226758,620,2011-12-31 23:38:36,2011-12-31 23:48:56,31103,31111,Member
1226757,484,2011-12-31 23:36:51,2011-12-31 23:44:55,31113,31202,Member


In [13]:
# your code here
pd.read_sql_query("""
    SELECT
      *
      FROM rides
      ORDER BY start_timestamp DESC
      LIMIT 10
""", db_connection)

,duration,start_timestamp,end_timestamp,start_station_number,end_station_number,member_type
0,468,2011-12-31 23:55:56,2012-01-01 00:03:45,31221,31111,Member
1,2060,2011-12-31 23:55:12,2012-01-01 00:29:33,31205,31222,Member
2,261,2011-12-31 23:47:27,2011-12-31 23:51:49,31107,31602,Member
3,387,2011-12-31 23:46:43,2011-12-31 23:53:10,31223,31201,Member
4,300,2011-12-31 23:41:19,2011-12-31 23:46:20,31201,31214,Member
5,349,2011-12-31 23:40:51,2011-12-31 23:46:40,31101,31203,Member
6,356,2011-12-31 23:40:39,2011-12-31 23:46:36,31101,31203,Member
7,410,2011-12-31 23:40:33,2011-12-31 23:47:23,31203,31214,Member
8,620,2011-12-31 23:38:36,2011-12-31 23:48:56,31103,31111,Member
9,484,2011-12-31 23:36:51,2011-12-31 23:44:55,31113,31202,Member


## Exercise 2
Get only the 'start_timestamp' and 'member_type' columns for the first 5 rides.

Note: you'll need to replace the `*`.

In [14]:
# Pandas way:
rides[['start_timestamp', 'member_type']].head(5)

,start_timestamp,member_type
0,2011-01-01 00:01:29,Member
1,2011-01-01 00:02:46,Casual
2,2011-01-01 00:06:13,Member
3,2011-01-01 00:09:21,Member
4,2011-01-01 00:28:26,Casual


In [15]:
# your code here
pd.read_sql_query("""
    SELECT
      start_timestamp, member_type
      FROM rides
      ORDER BY start_timestamp ASC
      LIMIT 5
""", db_connection)

,start_timestamp,member_type
0,2011-01-01 00:01:29,Member
1,2011-01-01 00:02:46,Casual
2,2011-01-01 00:06:13,Member
3,2011-01-01 00:09:21,Member
4,2011-01-01 00:28:26,Casual


# `query` / `filter`

`WHERE` lets you filter to include only certain rows

In [16]:
# pandas way:
(
    rides
    .query("start_station_number == 31620")
    [['start_station_number', 'start_timestamp', 'member_type']]
    .head(5)
)

,start_station_number,start_timestamp,member_type
0,31620,2011-01-01 00:01:29,Member
364,31620,2011-01-01 13:38:31,Member
413,31620,2011-01-01 14:16:43,Casual
415,31620,2011-01-01 14:17:03,Casual
483,31620,2011-01-01 14:51:20,Casual


In [17]:
# SQL way
pd.read_sql_query("""
    SELECT
      start_station_number, start_timestamp, member_type
      FROM rides
      WHERE start_station_number = 31620
      ORDER BY start_timestamp ASC
      LIMIT 5
""", db_connection)

,start_station_number,start_timestamp,member_type
0,31620,2011-01-01 00:01:29,Member
1,31620,2011-01-01 13:38:31,Member
2,31620,2011-01-01 14:16:43,Casual
3,31620,2011-01-01 14:17:03,Casual
4,31620,2011-01-01 14:51:20,Casual


## Exercise 3
Get the first 5 rides by Members.

Note:
* you'll need to `'quote'` strings, just like in Python---but the quotes should be *single* quotes (`'`) **not** double quotes (`"`)
* but unlike Python, the "equal" operator is just `=`, not `==`.

In [18]:
# your code here
pd.read_sql_query("""
    SELECT
      start_station_number, start_timestamp, member_type
      FROM rides
      WHERE member_type = 'Member'
      ORDER BY start_timestamp ASC
      LIMIT 5
""", db_connection)

,start_station_number,start_timestamp,member_type
0,31620,2011-01-01 00:01:29,Member
1,31400,2011-01-01 00:06:13,Member
2,31111,2011-01-01 00:09:21,Member
3,31605,2011-01-01 00:32:33,Member
4,31203,2011-01-01 00:35:48,Member


You can also get a single value by using an aggregation function, like `count`:

In [31]:
pd.read_sql_query("""
    SELECT
      COUNT(*)
      FROM rides
""", db_connection)

,COUNT(*)
0,1226767


# Exercise 3.2
Get the total number of rides from Members.

In [ ]:
# SQL way
# your code here
pd.read_sql_query("""
    SELECT
      COUNT(*)
      FROM rides
      WHERE member_type = 'Member'
""", db_connection)

# `grouping`

SQL gets powerful (and complicated) when you start grouping.

The basic template is: `SELECT` *things* `GROUP BY` *grouping expression*

*things* can include [**aggregation functions**](https://www.sqlite.org/lang_aggfunc.html) like `count`.

In [20]:
# Pandas way:
rides['member_type'].value_counts()

Member     979814
Casual     246949
Unknown         4
Name: member_type, dtype: int64

In [21]:
# get the number of rides by member type
pd.read_sql_query("""
    SELECT
      member_type, COUNT(*)
      FROM rides
      GROUP BY member_type
""", db_connection)

,member_type,COUNT(*)
0,Casual,246949
1,Member,979814
2,Unknown,4


In [22]:
# get the number of departures from each station
pd.read_sql_query("""
    SELECT
      start_station_number, COUNT(*)
      FROM rides
      GROUP BY start_station_number
      LIMIT 5
""", db_connection)

,start_station_number,COUNT(*)
0,31000,1171
1,31001,3492
2,31002,2883
3,31003,3432
4,31004,1598


It's often helpful to rename the results of aggregation functions. The `AS` keyword can rename any result column.

In [23]:
# get the number of departures from each station
pd.read_sql_query("""
    SELECT
      start_station_number, COUNT(*) AS num_departures
      FROM rides
      GROUP BY start_station_number
      LIMIT 5
""", db_connection)

,start_station_number,num_departures
0,31000,1171
1,31001,3492
2,31002,2883
3,31003,3432
4,31004,1598


## Exercise 4
Get the number of departures from each station, but only for Members.

In [24]:
# Pandas way:
(
    rides
    .query("member_type == 'Member'")
    .groupby('start_station_number')
    .size().to_frame('num_departures')
    .head(5)
)

,num_departures
start_station_number,
31000,869
31001,2666
31002,1989
31003,2345
31004,1370


In [25]:
# SQL way
# your code here
pd.read_sql_query("""
    SELECT
      start_station_number, COUNT(*) AS num_departures
      FROM rides
      WHERE member_type = 'Member'
      GROUP BY start_station_number
      LIMIT 5
""", db_connection)

,start_station_number,num_departures
0,31000,869
1,31001,2666
2,31002,1989
3,31003,2345
4,31004,1370


## Exercise 5
Break down those counts by member type. You'll need to:
* Include a column for `member_type`
* add `member_type` to the grouping expression

The result should look like:

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>start_station_number</th>
      <th>member_type</th>
      <th>num_departures</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>31000</td>
      <td>Casual</td>
      <td>302</td>
    </tr>
    <tr>
      <th>1</th>
      <td>31000</td>
      <td>Member</td>
      <td>869</td>
    </tr>
    <tr>
      <th>2</th>
      <td>31001</td>
      <td>Casual</td>
      <td>826</td>
    </tr>
    <tr>
      <th>3</th>
      <td>31001</td>
      <td>Member</td>
      <td>2666</td>
    </tr>
    <tr>
      <th>4</th>
      <td>31002</td>
      <td>Casual</td>
      <td>894</td>
    </tr>
  </tbody>
</table>

In [26]:
# your code here
df = pd.read_sql_query("""
    SELECT
      start_station_number, member_type, COUNT(*) AS num_departures
      FROM rides
      GROUP BY start_station_number, member_type
      LIMIT 5
""", db_connection)

## Exercise 6
Compute the number of rides per day.

Note: You'll need to use `date(start_timestamp)`, which is one of sqlite's [built-in date/time functions](https://www.sqlite.org/lang_datefunc.html)

In [27]:
# your code here
pd.read_sql_query("""
    SELECT
      date(start_timestamp) as start_day, COUNT(*) as num_rides
      FROM rides
      GROUP BY start_day
      LIMIT 5
""", db_connection)

,start_day,num_rides
0,2011-01-01,959
1,2011-01-02,781
2,2011-01-03,1301
3,2011-01-04,1536
4,2011-01-05,1571


# `join`

As before, let's mark which rides are on federal holidays. To do that, we'll use the `holidays` table. Notice how it's structured:

In [28]:
df = pd.read_sql_query("""
    SELECT
      *
      FROM holidays
""", db_connection)
print(len(df), "rows")
df.head()

49 rows


,date,is_holiday
0,2011-01-17,1
1,2011-02-21,1
2,2011-05-30,1
3,2011-07-04,1
4,2011-09-05,1


We'll need a `rides_by_date` table... (don't worry about this.)

In [29]:
pd.io.sql.execute("""DROP TABLE IF EXISTS rides_by_date""", db_connection)
pd.io.sql.execute("""CREATE TABLE rides_by_date AS SELECT
      date(start_timestamp) as start_day, COUNT(*) as num_rides
      FROM rides
      GROUP BY start_day""", db_connection);

Consider the example below. Think about why only 9 rows come out. Then:

* Change `JOIN` to `LEFT JOIN`; how many rows do you get then? Does that make sense?
* What values does `is_holiday` take on? Try replacing `is_holiday` by `IFNULL(is_holiday, 0)` (sqlite specific) or `COALESCE(is_holiday, 0)` (standard SQL). Does that achieve the result you hope for?
* Add an `AS is_holiday` clause to rename the `is_holiday` column to a more useful name.

(You don't need to include textual answers, just change the code as directed.)

In [30]:
df = pd.read_sql_query("""
    SELECT
      start_day, num_rides, COALESCE(is_holiday, 0)
      FROM rides_by_date
      LEFT JOIN holidays ON rides_by_date.start_day = holidays.date
""", db_connection)
print(len(df), "rows")
df.head(n=31)

365 rows


,start_day,num_rides,"COALESCE(is_holiday, 0)"
0,2011-01-01,959,0
1,2011-01-02,781,0
2,2011-01-03,1301,0
3,2011-01-04,1536,0
4,2011-01-05,1571,0
5,2011-01-06,1573,0
6,2011-01-07,1493,0
7,2011-01-08,939,0
8,2011-01-09,802,0
9,2011-01-10,1298,0
